In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from urllib.request import urlopen
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [36]:
path = '/Users/yichuanhong/Documents/Python Project/Vietnam Avian Flu Outbreak Research/NLP/chromedriver'
url = 'https://baonghean.vn/search/Y8O6bSBnaWEgY-G6p20=/cum-gia-cam.html'
root = 'https://baonghean.vn'
df = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])
driver = webdriver.Chrome(path)
driver.get(url)

In [37]:
for i in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
print('Reached bottom of page')

Reached bottom of page


In [38]:
soup = BeautifulSoup(driver.page_source)
news = soup.find_all('article', class_ = 'story story--horizontal')
for item in news:
    title = item.find('h2', class_ = 'story__title').text
    date = item.find('time')['datetime']
    date = datetime.strptime(date, '%Y-%m-%d')
    sub_url = item.find('h2', class_ = 'story__title').find('a')['href']
    if sub_url.startswith('https'):
        url = sub_url
    else:
        url = root + sub_url
    response = requests.get(url).text
    new_soup = BeautifulSoup(response, 'lxml')
    content = new_soup.find('div', class_ = 'content cms-body')
    if content:
        content = content.text
    new_row = {'Title': title, 'URL': url, 'Date': date, 'Content': content}
    df = df.append(new_row, ignore_index = True)

In [39]:
news = soup.find_all('article', class_ = 'story story--horizontal exclusive')
for item in news:
    title = item.find('h2', class_ = 'story__title').text
    date = item.find('time')['datetime']
    date = datetime.strptime(date, '%Y-%m-%d')
    sub_url = item.find('h2', class_ = 'story__title').find('a')['href']
    if sub_url.startswith('https'):
        url = sub_url
    else:
        url = root + sub_url
    response = requests.get(url).text
    new_soup = BeautifulSoup(response, 'lxml')
    content = new_soup.find('div', class_ = 'content cms-body long-form')
    if content:
        content = content.text
    new_row = {'Title': title, 'URL': url, 'Date': date, 'Content': content}
    df = df.append(new_row, ignore_index = True)

In [40]:
df

,Title,URL,Date,Content
0,"\nNghệ An ra công điện khẩn phòng, chống chủng...",https://baonghean.vn/nghe-an-ra-cong-dien-khan...,2021-07-15,\nUBND tỉnh Nghệ An vừa ra Công điện khẩn về v...
1,\nNghệ An: Dịch tả lợn châu Phi và cúm gia cầm...,https://baonghean.vn/nghe-an-dich-ta-lon-chau-...,2021-03-02,\nTheo báo cáo của cơ quan chức năng tỉnh Nghệ...
2,\nCông điện khẩn về triển khai các giải pháp k...,https://baonghean.vn/cong-dien-khan-ve-trien-k...,2021-02-26,"\nTheo thông báo của Cục Thú y, từ đầu năm 202..."
3,"\nCông điện khẩn về tập trung kiểm soát, phòng...",https://baonghean.vn/cong-dien-khan-ve-tap-tru...,2021-01-19,\r\n Nội dung công điện...
4,\nThêm trại vịt ở Yên Thành nhiễm dịch cúm gia...,https://baonghean.vn/them-trai-vit-o-yen-thanh...,2020-09-06,"\n\nThêm trại vịt trên địa bàn xã Nam Thành, h..."
...,...,...,...,...
301,"\nThiếu cán bộ thú y trong phòng, chống dịch b...",https://baonghean.vn/thieu-can-bo-thu-y-trong-...,2020-03-03,\nXã Quỳnh Hồng (Quỳnh Lưu) phải thuê đội ngũ ...
302,\nNghệ An: Công nghiệp tiếp tục tăng trưởng nh...,https://baonghean.vn/nghe-an-cong-nghiep-tiep-...,2020-02-21,"\nSáng 21/2, UBND tỉnh tổ chức phiên họp thườn..."
303,\n24 xã của 10 huyện ở Nghệ An còn có dịch tả ...,https://baonghean.vn/24-xa-cua-10-huyen-o-nghe...,2020-02-14,\n\n\nTrước diễn biến phức tạp của các loại bệ...
304,\nNghệ An yêu cầu các địa phương huy động cả h...,https://baonghean.vn/nghe-an-yeu-cau-cac-dia-p...,2020-02-12,\nHuy động cả hệ thống chính trị vào cuộc\nTrê...


In [41]:
df.to_csv('data/baonghean_articles_vietnamese.csv', index = False)

In [42]:
driver.quit()